# DWD project - Team Anaconda

### file 1 - NYC Housing Data

### Installing packages outside the default library and importing necessary libraries

In [ ]:
#importing packages required for this python notebook. The most up to date panda package is needed
import pandas as pd
import os
import numpy as np
import matplotlib as plt
import datetime as dt

### Pulling in and combining all the datasets into one dataframe

The files from the nyc.gov site are separated into year in borough. We are running 8 years of data so there are 40 different files being pulled into the dataframe. The files need to be in the same folder as this notebook. This will take about 2 minutes to run.

In [ ]:
path = os.getcwd()
files = os.listdir(path)

files_xls = [f for f in files if f[-3:] == 'xls'] # the housing data files are all .xls, so we are searching the current folder for all .xls

nyc_housing_data = pd.DataFrame()

for f in files_xls:
    if f[3] == '0':         # need to create a certain rule around the 2010 files
        data = pd.read_excel(f, header=3, index_col=None) # the column names for the files 2011 - 2017 start in row 5, for 2010 it starts in row 4
        data.rename(columns=lambda x: x.strip(), inplace= True)
        nyc_housing_data = nyc_housing_data.append(data, sort = False)
    else:
        data = pd.read_excel(f, header=4, index_col=None)
        data.rename(columns=lambda x: x.strip(), inplace= True)
        nyc_housing_data = nyc_housing_data.append(data, sort = False)

### Getting just the necessary columns into the dataframe

In [ ]:
# getting the specs of the dataframe
nyc_housing_data.shape

In [ ]:
# getting the column name and data type for the new combined dataframe
nyc_housing_data.info()

In [ ]:
# creating a column within the dataframe just with the year of sale
nyc_housing_data['SALE YEAR'] = nyc_housing_data['SALE DATE'].dt.year

In [ ]:
# creating a new df with only columns needed
nyc_housing_data_clean = nyc_housing_data.iloc[:,[0,1,2,8,10,11,12,14,15,16,18,19,20,22,23]]

In [ ]:
# getting specs of new data frame
nyc_housing_data_clean.info()

### Eliminating rows to get the true population of residential housing sales

In [ ]:
# removing any sale prices that are less than $50k as it is most likely not "real" sales
nyc_housing_data_clean = nyc_housing_data_clean[nyc_housing_data_clean['SALE PRICE'] > 50000]


In [ ]:
# getting the specs of the dataframe after removing the above rows
nyc_housing_data_clean.shape

In [ ]:
#leaving only residential properties of 4 or less units
nyc_housing_data_clean = nyc_housing_data_clean[nyc_housing_data_clean['COMMERCIAL UNITS'] < 1] 
nyc_housing_data_clean = nyc_housing_data_clean[nyc_housing_data_clean['RESIDENTIAL UNITS'] < 5]


In [ ]:
# even after removing the above, there still may be properties that are not residential
# using the 'BUILDING CLASS CATEGORY' to see what other type of properties are in the dataset
nyc_housing_data_clean['BUILDING CLASS CATEGORY'].unique()

In [ ]:
# The 'BUILDING CLASS CATEGORY' has unnecessary spaces within the data
# running ocde to remove those spaces
nyc_housing_data_clean['BUILDING CLASS CATEGORY'] = nyc_housing_data_clean['BUILDING CLASS CATEGORY'].map(lambda x: x.strip())

In [ ]:
#checking if the code removed the necessary spaces
nyc_housing_data_clean['BUILDING CLASS CATEGORY'].unique()

In [ ]:
# creating a list of properties that would not be classified as residential properties
# removing those rows from the dataset
remove_list = ["05 TAX CLASS 1 VACANT LAND","05 TAX CLASS 1 VACANT LAND","06 TAX CLASS 1 - OTHER","06 TAX CLASS 1 - OTHER","11 SPECIAL CONDO BILLING LOTS","11A CONDO-RENTALS","16 CONDOS - 2-10 UNIT WITH COMMERCIAL UNIT","16 CONDOS - 2-10 UNIT WITH COMMERCIAL UNIT","18 TAX CLASS 3 - UNTILITY PROPERTIES","18 TAX CLASS 3 - UTILITY PROPERTIES","21 OFFICE BUILDINGS","21 OFFICE BUILDINGS","22 STORE BUILDINGS","22 STORE BUILDINGS","23 LOFT BUILDINGS","25 LUXURY HOTELS","26 OTHER HOTELS","26 OTHER HOTELS","27 FACTORIES","27 FACTORIES","28 COMMERCIAL CONDOS","28 COMMERCIAL CONDOS","29 COMMERCIAL GARAGES","29 COMMERCIAL GARAGES","30 WAREHOUSES","30 WAREHOUSES","31 COMMERCIAL VACANT LAND","31 COMMERCIAL VACANT LAND","32 HOSPITAL AND HEALTH FACILITIES","32 HOSPITAL AND HEALTH FACILITIES","33 EDUCATIONAL FACILITIES","33 EDUCATIONAL FACILITIES","34 THEATRES","34 THEATRES","35 INDOOR PUBLIC AND CULTURAL FACILITIES","35 INDOOR PUBLIC AND CULTURAL FACILITIES","36 OUTDOOR RECREATIONAL FACILITIES","36 OUTDOOR RECREATIONAL FACILITIES","37 RELIGIOUS FACILITIES","37 RELIGIOUS FACILITIES","38 ASYLUMS AND HOMES","39 TRANSPORTATION FACILITIES","41 TAX CLASS 4 - OTHER","41 TAX CLASS 4 - OTHER","42 CONDO CULTURAL/MEDICAL/EDUCATIONAL/ETC","42 CONDO CULTURAL/MEDICAL/EDUCATIONAL/ETC","43 CONDO OFFICE BUILDINGS","43 CONDO OFFICE BUILDINGS","44 CONDO PARKING","44 CONDO PARKING","45 CONDO HOTELS","45 CONDO HOTELS","46 CONDO STORE BUILDINGS","46 CONDO STORE BUILDINGS","47 CONDO NON-BUSINESS STORAGE","47 CONDO NON-BUSINESS STORAGE","48 CONDO TERRACES/GARDENS/CABANAS","48 CONDO TERRACES/GARDENS/CABANAS","49 CONDO WAREHOUSES/FACTORY/INDUS","49 CONDO WAREHOUSES/FACTORY/INDUS"]
nyc_housing_data_clean = nyc_housing_data_clean[~nyc_housing_data_clean['BUILDING CLASS CATEGORY'].isin(remove_list)]

In [ ]:
# getting the specs of the dataframe after removing the above rows
nyc_housing_data_clean.shape

In [ ]:
#removing any rows with no zip code
nyc_housing_data_clean = nyc_housing_data_clean[nyc_housing_data_clean['ZIP CODE'] >0] 
nyc_housing_data_clean.shape

### Creating a data file that has median housing info from the dataset

In [ ]:
# now that the data is cleaned by showing residential properties only we need to create a table with the median property value for each zip code for each year
# then exporting that table to a csv

nyc_housing_data_summary=nyc_housing_data_clean.groupby(['ZIP CODE','SALE YEAR'], as_index=False)['SALE PRICE'].median()
nyc_housing_data_summary2=nyc_housing_data_clean.groupby(['ZIP CODE','SALE YEAR'], as_index=False)['SALE PRICE'].count()
nyc_housing_data_summary['Sales Count'] = nyc_housing_data_summary2['SALE PRICE'].values
nyc_housing_data_summary.rename(columns={'SALE PRICE': 'Median Sale Price'}, inplace=True)
nyc_housing_data_summary.to_csv("C:/Users/owner/Downloads/DWD/Final_Median_Housing_Data.csv",index=False)